## Cost Analysis of Eggs
# 5- year time span, conventionally and organically produced

In [1]:
# Load the csv file

In [2]:
# Drop unneeded columns. Combine the Organic rows into one organic categroy. Create a dataframe. 

In [3]:
# Group the data by date on month. Depending on how the date fields are organized, I might need to 
# extract the month from the date field. 

In [4]:
# Create a line graph of the change in cost in eggs over 5 years. 